In [1]:
# import 
import sys; 
if '../../modules/' not in sys.path: sys.path.append('../../modules/')
import myutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

# fast.ai import
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor

In [2]:
path = '../../../datasets/grocery/'

In [3]:
!head -3 {path}train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,


In [4]:
!ls {path}

holidays_events.csv  sample_submission.csv.7z  train.csv
items.csv	     stores.csv		       Train.pickle
oil.csv		     test.csv		       transactions.csv


In [5]:
!head -6 {path}items.csv {path}stores.csv {path}transactions.csv {path}oil.csv {path}holidays_events.csv

==> ../../../datasets/grocery/items.csv <==
item_nbr,family,class,perishable
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1

==> ../../../datasets/grocery/stores.csv <==
store_nbr,city,state,type,cluster
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4

==> ../../../datasets/grocery/transactions.csv <==
date,store_nbr,transactions
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922

==> ../../../datasets/grocery/oil.csv <==
date,dcoilwtico
2013-01-01,
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.2

==> ../../../datasets/grocery/holidays_events.csv <==
date,type,locale,locale_name,description,transferred
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Funda

In [6]:
#%%time
#df_all = pd.read_csv(f'{path}train.csv', date_parser=['date']) 

    <string>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.

    CPU times: user 56.1 s, sys: 7.8 s, total: 1min 3s
    Wall time: 7min 15s

**To make this fast, better if can define the data type of columns at first hand, so that python have to doo less work**

In [7]:
types= {'id':'int64','store_nbr':'int8','item_nbr':'int32','unit_sales':'float32','onpromotion':'object'}

In [8]:
# %%time
# df_all = pd.read_csv(f'{path}train.csv', date_parser=['date'], dtype=types, low_memory=False, infer_datetime_format=True) 

In [9]:
# %%time
# df_train = pd.read_csv(f'{path}train.csv', parse_dates=['date'], dtype=types, low_memory=False, infer_datetime_format=True) 

In [10]:
# pd.to_pickle(df_all, f'{path}Train.pickle')  #took less space

In [11]:
# df_all.to_feather(f'{path}Train.feather')  # not better compared with pickle

In [12]:
df_all = pd.read_pickle(f'{path}Train.pickle')

In [14]:
df_all.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [15]:
df_all.isnull().sum()

id                    0
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
onpromotion    21657651
dtype: int64

In [16]:
df_all.onpromotion.fillna(False, inplace = True)
df_all.onpromotion = df_all.onpromotion.map({'False':False, 'True':True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

In [17]:
%time  df_all.describe(include='all')

CPU times: user 45.3 s, sys: 9.62 s, total: 54.9 s
Wall time: 45.7 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554856e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN
50%,6.274852e+07,NaN,2.800000e+01,9.595000e+05,4.000000e+00,NaN
75%,9.412278e+07,NaN,4.300000e+01,1.354380e+06,9.000000e+00,NaN
